In [2]:
# 引用库
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List
import os
import hashlib
import inspect
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Tokenization

### 简介
Tokenization 的主要目的是将文本分解成更小的单位(Tokens)，减小模型输入数据的内在结构复杂度(从句子变为单词序列)，从而简化模型训练的难度。同时将字符的序列转化为Token序号的序列，便于模型输入。

Tokenization 首先确定语言的词表划分粒度，一般可分为：
* 字符级：将文本分解为字符。
* 单词级：将文本分解为单词。
* 子词级：将单词进一步分解为更小的有意义单元（如前缀、后缀）。

之后使用预定义的规则来识别 tokens, 或使用统计或机器学习技术来识别最优的 token 切分方式。例如，BPE（Byte Pair Encoding）或 SentencePiece。

最后实现一组文本序列和Tokens序列之间相互转化的函数，即可完成Tokenization部分。

### 实验要求

1. 实现字符级切分的简单tokenizer， 由 字符表， 字符到token的 encoder()函数 和 token到字符的 decoder() 函数组成。
2. 调用 现有的tokenizer实现，比如openai 的tiktoken

In [3]:
class Tokenizer:
    def __init__(self, dataPath:str):
        with open(dataPath,"r",encoding="utf-8") as f:
            self.dataset = f.read()
        self.generate_vocabulary()
    
    # 产生符号表
    def generate_vocabulary(self, ):
        self.char2index = {}
        self.index2char = {}

        # 添加开始和结束符号
        self.char2index["<START>"] = 0
        self.char2index["<END>"] = 1
        self.index2char[0] = "<START>"
        self.index2char[1] = "<END>"

        # 对数据集中的每个字符都标号，从2开始
        index = 2 
        for char in sorted(set(self.dataset)):
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index] = char
                index += 1

    # 对数据集进行编码
    def encode(self, sentence : str) -> torch.Tensor:
        # 添加开始符号
        tokenflow = [self.char2index["<START>"]]
        # 对于输入句子的每个字符，依次编号添加到tokens中
        # 如果有不在符号表中的字符，用终结符替代
        for char in sentence:
            tokenflow.append(self.char2index.get(char, self.char2index["<END>"]))  
        # 添加终结符号
        tokenflow.append(self.char2index["<END>"])
        # 以Tensor形式返回  
        return torch.tensor(tokenflow, dtype=torch.long)
    
    # 编码后进行解码
    def decode(self, tokens : torch.Tensor) -> str:
        # 转换为list
        if isinstance(tokens, torch.Tensor):
            tokens = tokens.tolist()
        # 解码为字符流
        charflow = []
        for token in tokens:
            # 未知编码返回<UNK>即Unknown
            char = self.index2char.get(token, "<UNK>")
            if char not in ["<START>", "<END>"]:
                charflow.append(char)
        # 连接成字符串返回
        return "".join(charflow)



### 定义 dataloader 和 dataset

为了高效加载数据，我们需要把输入文件接入 PyTorch 的数据加载器中。在这里我们定义 `ShakespeareDataset` 类用于加载数据集，用 PyTorch 的 `DataLoader` 类来实现数据加载。


In [4]:
class ShakespeareDataset(Dataset):
    def __init__(self, filepath, tokenizer, chunk_size):
        self.tokenizer = tokenizer
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        self.encoded = self.tokenizer.encode(text)
        self.chunk_size = chunk_size

    def __len__(self):
        return len(self.encoded) - self.chunk_size

    def __getitem__(self, idx):
        # 提取文本，文本作为输入，每个字符的下一个字符作为标签（size+1）
        chunk = self.encoded[idx: idx + self.chunk_size]
        label = self.encoded[idx + 1: idx + self.chunk_size + 1]
        return chunk, label

tokenizer = Tokenizer(dataPath="input.txt")

def create_dataloader(filepath, tokenizer, chunk_size, batch_size, shuffle=True):
    dataset = ShakespeareDataset(filepath, tokenizer, chunk_size)
    train_dataset,val_dataset = torch.utils.data.random_split(dataset,[int(len(dataset)*0.8),len(dataset)-int(len(dataset)*0.8)])
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
    return train_dataloader, val_dataloader

train_dataloader,val_dataloader = create_dataloader('input.txt', tokenizer, chunk_size=200, batch_size=2)


注意力的计算公式为：
$$
Head = Attention(x)=Softmax(M\cdot QK^T)V\\
Q=xW_{q},K=xW_{k}, V=xW_{v}
$$
这里实现的一些数学技巧可以参见attention.ipynb

In [ ]:
class HeadAttention(nn.Module):
    def __init__(self, seq_len:int, embed_size:int, hidden_size:int):
        super().__init__()

        self.register_buffer("tril", torch.tril(torch.ones(seq_len, seq_len)))
        
        # 初始化三个矩阵
        self.to_q = nn.Linear(embed_size, hidden_size, bias=False)
        self.to_k = nn.Linear(embed_size, hidden_size, bias=False)
        self.to_v = nn.Linear(embed_size, hidden_size, bias=False)

    def forward(self, inputs):
        batch_size, seq_len, embed_size = inputs.shape

        # 依次计算queries,keys,values
        queries = self.to_q(inputs) 
        keys = self.to_k(inputs)  
        values = self.to_v(inputs)  

        d_k = queries.shape[-1]
        queries = queries / (d_k ** 0.5)
        # 计算注意力得分
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) 
        # 实施掩码
        mask = self.tril[:seq_len, :seq_len]
        attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
        # 计算注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1)  
        # 得到输出
        attention_output = torch.matmul(attention_weights, values)

        return attention_output  

Transformer中使用的注意力机制时会使用多个注意力头，期望每个注意力头能够注意到不同的信息。
所以实际公式需要修改如下
$$
MultiHeadAttention(x)=[Head_0, Head_1,...,Head_h]W_o\\
Head_i = Attention(x)=Softmax(M\cdot Q_iK_i^T)V_i\\
Q_i=xW_{iq},K=xW_{ik}, V=xW_{iv}
$$
在搭建网络的过程中，同学们可能会用到nn.ModuleList这个库，每个$Head_i$的计算可以直接使用上面已经实现的单头注意力计算。
最后对于这些注意力头再使用一个简单的线性层/矩阵$W_o$汇总信息即可

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads:int, head_size:int, seq_len:int, embed_size:int):
        super().__init__()
        self.n_heads = n_heads
        # 计算头的大小
        self.head_size = embed_size // n_heads  
        # 确保整除，分割正确
        assert embed_size % n_heads == 0, "attention:embed_size should divide n_heads!"
        # 建立多头注意力层
        self.heads = nn.ModuleList([HeadAttention(seq_len, embed_size, self.head_size) for _ in range(n_heads)])
        self.projection = nn.Linear(embed_size, embed_size)


    def forward(self, inputs):
        batch_size, seq_len, embed_size = inputs.shape
        # 确保匹配
        assert embed_size == self.n_heads * self.head_size, "embed_size must be equal to n_heads * head_size"
        # 计算每个头的输出
        outputs = [head(inputs) for head in self.heads] 
        # 链接所有的头输出
        connect_output = torch.cat(outputs, dim=-1) 
        # 回到最初的大小
        output = self.projection(connect_output)  
        return output  

## 专家网络 Expert

Expert即为标准Transformer中的FeedForward模块。

在经过MultiHeadAttention 模块后，seq_len中的每一个Embedding都对应了前文信息的加权求和。在经过FeedForward模块时，模型对每一个位置的Embedding进行了两次线性变换和一次非线性变换，可以视为对当前语境下的信息进行加工。知识编辑的一些研究表明，FeedForword 模块参数包含了大量的事实性知识。

一个直观的想法是，类比于MultiHeadAttention，我们在每一层训练多个FeedForward模块，对于不同位置的Embedding使用不同的FeedForward模块处理对应的信息。就好像每层有多个Expert,每个Expert都负责处理一类数据的深加工，因此我们称FeedForward为Expert。

实现方面:

FeedForward层由两层简单的线性层组成，对于一个(batch_size, seq_len, embed_size)输入的向量x
只在最后一个维度上进行计算，以实现词的特征维度上的交互(注意力机制是词之间的交互)。
其首先用一个线性层将x最后一维扩大至原先4倍，然后继续用一个线性层还原回原先的维度。

In [ ]:
# 专家层
class Expert(nn.Module):
    def __init__(self, embed_size: int):
        super().__init__()
        # 初始化两个线性层
        self.fc1 = nn.Linear(embed_size, 4 * embed_size)
        self.fc2 = nn.Linear(4 * embed_size, embed_size)

    def forward(self, inputs):
        swap = self.fc1(inputs) 
        # 实施非线性激活
        swap = F.relu(swap) 
        outputs = self.fc2(swap) 
        return outputs

## 选通网络 TopkRouter

在实现了单个Expert后，我们要设计一个选通网络决策每个Embedding要使用那个Expert计算


### 为了说明选通网络的实现方式，我们定义一下记号：

inputs.shape = [batch_size, seq_len, embed_size] = [1, 8, 16] 

即输入有batch_size=1个数据点，该数据有seq_len长度的context，即包含seq_len=8个Embedding，每个Embedding长度为embed_dim=16。

记 num_expert = 4, 即该层包含 num_expert 个并列的Expert。

记 active_expert = 2, 即计算每个Embedding仅有 active_expert 个Expert 参与计算。

### 选通网络计算
对于有seq_len=8的数据，如果每个Expert都参与计算每一个Embedding，那么一共需要计算 seq_len*embed_size = 32 次， 这极大的增加了模型计算量，因此我们往往只激活其中的active_experts个Expert，这要求我们对每一个Embedding计算最合适的active_experts个 Expert。

对于单个Expert 的原版Transformer来说：

$$
outputs[0,seq] = FeedForward(inputs[0,seq])
$$

对于多个Expert的网络：

$$
outputs[0,seq] = \sum_{i \in range(num\_model)} \alpha_{i} Expert_{i}(inputs[0,seq])
$$

$$
\alpha_{i} = \left\{
\begin{array}{ll}
    1 & Expert_{i}  \text{is selected} \\
    0 & Expert_{i}  \text{is not selected} \\
\end{array}
\right.
$$
将$\{\alpha_0,\alpha_1,\dots,\alpha_{num_experts-1}\}$记为向量$\alpha$:
$$
outputs[0,seq] = \alpha \cdot \{Expert_i(inputs[0,seq])\}
$$

一个选通0,2号Expert的$\alpha$的例子是$[1,0,1,0]$

问题在于如何求得 $\alpha$, 对于一个Embedding ，我们使用神经网络对每个Expert打分，在根据分数计算$\alpha$

$$
score[0,seq] = MLP(inputs[0,seq])  \\
\alpha = topK(score[0,seq])
$$

例如：

$$
score[0,seq] = [11.32,1.54,14.83,-1.90] \\
\alpha = [1,0,1,0]
$$

从优化的角度来说，$\alpha$取前k大的分数的下标（即argmax），这个操作是不可导的，这里我们用之前在"attention.ipynb"中提到的技巧处理这里的计算。

$$
mask(score[0,seq]) = [11.32,-inf,14.83,-inf] \\
\alpha = softmax(mask(score[0,seq])) = [0.028,0,0.971,0] \\
index = [1,0,1,0]
$$

我们用这个$\alpha$和$index$用做选通网络.

In [ ]:
# 
class TopkRouter(nn.Module):
    def __init__(self, embed_size, num_experts, active_experts):
        super().__init__()
        self.num_experts = num_experts
        self.active_experts = active_experts
        # 对专家层进行线性变换
        self.router_weights = nn.Linear(embed_size, num_experts)
    
    def forward(self, inputs):
        batch_size, seq_len, embed_size = inputs.shape
        # 为每个token计算logits
        logits = self.router_weights(inputs)
        # 实施softmax得到常规化结果
        router_output = F.softmax(logits, dim=-1) 
        # 对于每个词token，选定前k个有最高权重的专家层
        topk_values, indices = torch.topk(router_output, self.active_experts, dim=-1)
        # 常规化选定的专家层
        topk_normalized = topk_values / topk_values.sum(dim=-1, keepdim=True)
        return topk_normalized, indices

## 稀疏专家网络 SparseMoE

![moe](./moeSparse.png)

在定义完Expert 和 TopkRouter后，我们可以定义SparseMoE模块。

在前向过程中，对于inputs.shape = [Batch_size,seq_len,embed_size]第二维度seq_len个Embedding,我们先利用TopkRouter计算出选通专家序号indices以及专家权重router_output。

我们将Embedding通过选通的Expert得出active_expert个新的Embedding，然后使用router_output的作为权重对新的Embedding加权求和作为输出。


In [ ]:
class SparseMoE(nn.Module):
    def __init__(self, embed_size: int, num_experts: int, active_experts: int):
        super().__init__()
        self.num_experts = num_experts
        self.active_experts = active_experts
        # 初始化router
        self.router = TopkRouter(embed_size, num_experts, active_experts)
        # 初始化专家层
        self.experts = nn.ModuleList([Expert(embed_size) for _ in range(num_experts)])

    def forward(self, inputs):
        batch_size, seq_len, embed_size = inputs.shape
        router_output, indices = self.router(inputs)
        # 最终输出
        final_output = torch.zeros_like(inputs)

        for i in range(self.active_experts):
            expert_idx = indices[:, :, i]
            # 准备掩码
            mask = torch.zeros(batch_size, seq_len, self.num_experts, device=inputs.device)
            mask.scatter_(2, expert_idx.unsqueeze(-1), 1)
            # 从现在的专家收集输入
            selected_inputs = inputs.unsqueeze(2) * mask.unsqueeze(-1)
            selected_inputs = selected_inputs.sum(dim=2) 
            # 对选定的输入实施专家层
            expert_output = self.experts[i](selected_inputs)  
            # 计算输出
            final_output += expert_output * router_output[:, :, i].unsqueeze(-1) 
        return final_output


Transformer由一层层的block堆叠而成，其中每个block的结构从模型的结构图展开中可以看到，由LayerNorm，Masked multi head attention，(SparseMoE)FeedForward组成。

对于一个表示句子的输入向量x，其首先会经过Layer Normalization层.
Layer Normalization 层对于一个 句子个数x句子长度x单词向量维度 的输入 x, 会在最后两维上进行规范化处理，起到稳定训练的作用。

$$
LN(x)=\frac{x-mean(x)}{\sqrt{var(x)+\epsilon}}\cdot\gamma+\beta
$$

其中mean和var都是在最后两个维度上进行的，layernorm的实现同学们可以直接调用nn.LayerNorm
经过layernorm层后，再经过Mask multi head attention层之后，会在+号处再次和原始的输入进行相加，这样的做法能够提高训练的稳定性。有兴趣的同学可以从梯度角度思考原因，或者搜索残差连接相关资料进行学习。
之后再同样经过一层layernorm和feedforwad之后，就可以得到block块的输出了。
即 x' = x+MHA(LN(x)), y = FFN(LN(x'))+x'

In [ ]:
class Block(nn.Module):
    def __init__(self, embed_size: int, n_heads: int, seq_len: int, num_experts: int, active_experts: int):
        super().__init__()
        # 多头注意层
        self.attention = MultiHeadAttention(n_heads, embed_size // n_heads, seq_len, embed_size)
        # SparseMoE
        self.sparse_moe = SparseMoE(embed_size, num_experts, active_experts)
        # 前馈层
        self.feed_forward = nn.Sequential(nn.Linear(embed_size, 4 * embed_size), nn.ReLU(), nn.Linear(4 * embed_size, embed_size))
        # 常规化层
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)
        self.ln3 = nn.LayerNorm(embed_size)

    def forward(self, inputs):
        # 多头注意力
        attn_output = self.attention(inputs)
        attn_output = self.ln1(inputs + attn_output) 
        # SparseMoE
        moe_output = self.sparse_moe(attn_output)
        moe_output = self.ln2(attn_output + moe_output)
        # 前馈层
        ff_output = self.feed_forward(moe_output)
        output = self.ln3(moe_output + ff_output) 

        return output

In [ ]:
class SparseMoETransformer(nn.Module):
    def __init__(self, vocab_size:int, seq_len:int, embed_size:int, n_layers:int, n_heads:int, num_experts:int, active_experts:int):
        super().__init__()

        self.seq_len = seq_len
        self.embed_size = embed_size
        # 词汇嵌入、位置编码
        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(seq_len, embed_size)
        # Transformer块
        self.blocks = nn.ModuleList([Block(embed_size, n_heads, seq_len, num_experts, active_experts) for _ in range(n_layers)])
        # 层常规化并且输出
        self.layer_norm = nn.LayerNorm(embed_size)
        self.output_layer = nn.Linear(embed_size, vocab_size)

    def forward(self, inputs, labels=None):
        batch_size, seq_len = inputs.shape
        # 为每个位置创建id
        position_ids = torch.arange(seq_len, device=inputs.device).unsqueeze(0).expand(batch_size, -1)
        # 嵌入
        token_embeddings = self.token_embedding(inputs)
        position_embeddings = self.position_embedding(position_ids)  
        embeddings = token_embeddings + position_embeddings  
        # 从Transfromer块传过去
        x = embeddings
        for block in self.blocks:
            x = block(x)
        x = self.layer_norm(x)
        logits = self.output_layer(x)
        # 如果有标签，计算损失
        if labels is None:
            loss = None
        else:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
        return logits, loss
    


    def generate(self, inputs, max_new_tokens):
        encoded_input = tokenizer.encode(inputs)
        encoded_input = torch.tensor(encoded_input).unsqueeze(0) 
        device = next(self.parameters()).device  
        encoded_input = encoded_input.to(device)

        if encoded_input.size(1) > self.seq_len:
            encoded_input = encoded_input[:, :self.seq_len]

        generated = encoded_input
        for _ in range(max_new_tokens):
            if generated.size(1) > self.seq_len:
                generated_input = generated[:, -self.seq_len:]
            else:
                generated_input = generated

            logits, _ = self.forward(generated_input)
            last_logits = logits[:, -1, :]
            next_token_ids = torch.argmax(last_logits, dim=-1)
            next_token_ids = next_token_ids.unsqueeze(-1)
            generated = torch.cat([generated, next_token_ids], dim=1)

        generated_ids = generated[0].tolist()
        generated_text = tokenizer.decode(generated_ids)
        return generated_text

# 训练循环

如果你已经完成了模型定义等内容，训练的过程实际上在高度封装的 Pytorch 库中非常简单, 因为你并不需要写对应的反向传播。



#### Loss 

Loss 用来**衡量**模型预测与真实值之间的**差距**。

常见的几个 Loss 函数：

* 交叉熵：$\text{CrossEntropy Loss} = -\sum_{i=1}^{n} y_i \log(\hat{y}_i)$
* 均方误差：$\text{MSE} = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2$
* 绝对误差：$\text{MAE} = \frac{1}{n}\sum_{i=1}^{n} |y_i - \hat{y_i}|$

不同的 loss 对应不同的优化目标，如果写错 loss 函数会导致模型不收敛/性能很差。

#### 训练循环

当我们写好 Optimizer 和 Loss 之后，对应的训练循环就十分简单了。

我们只需要做以下事情：

* 从 dataloader 里面拿到一个 batch 的数据以及标签
* 将数据送入模型，进行前向传播
* 拿到模型输出的 logits
* 将 logits 和 标签进行 loss 计算
* 用 Optimizer 
    * 清空梯度
    * 反向传播
    * 更新参数

In [ ]:
def train(model, dataloader, epoch, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model.train()
    total_loss = 0
    for i, (inputs, targets) in tqdm(enumerate(dataloader), total=len(dataloader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        logits, loss = model(inputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch} Loss: {total_loss / len(dataloader)}')
    return total_loss / len(dataloader)

def validate(model, dataloader, epoch, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(dataloader), total=len(dataloader)):
            inputs, targets = inputs.to(device), targets.to(device)
            logits, loss = model(inputs, targets)
            total_loss += loss.item()
    print(f'Epoch {epoch} Validation Loss: {total_loss / len(dataloader)}')
    return total_loss / len(dataloader)

In [ ]:
# 创建数据加载器
train_dataloader, valid_dataloader = create_dataloader('input.txt', tokenizer, chunk_size=50, batch_size=512)

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SparseMoETransformer(vocab_size=len(tokenizer.char2index), seq_len=50, embed_size=64, n_layers=3, n_heads=8, num_experts=8, active_experts=2
                            ).to(device)

# 训练模型
def run(model, train_dataloader, valid_dataloader, device, epochs=10):
    train_losses = []
    valid_losses = []
    for epoch in range(epochs):
        train_loss = train(model, train_dataloader, epoch, device)
        valid_loss = validate(model, valid_dataloader, epoch, device)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        print(f'Epoch {epoch} Train Loss: {train_loss}, Valid Loss: {valid_loss}')

    # 绘图输出损失
    plt.figure(figsize=(10, 5))
    plt.plot(range(epochs), train_losses, label='Train Loss')
    plt.plot(range(epochs), valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.show()

# 运行，训练轮次依次选择5,10,20，观察不同效果
run(model, train_dataloader, valid_dataloader, device, epochs=20)
# 保存模型
torch.save(model.state_dict(), 'model.pth')
# 加载模型
model.load_state_dict(torch.load('model.pth'))

# 测试生成文本，最多300词，选择五个句子进行测试
generated_text = model.generate("I have a dog.", tokenizer=tokenizer,max_new_tokens=300)
print(generated_text)

generated_text = model.generate("You are so fool!", tokenizer=tokenizer,max_new_tokens=300)
print(generated_text)

generated_text = model.generate("Where is the God?", tokenizer=tokenizer,max_new_tokens=300)
print(generated_text)

generated_text = model.generate("How are you?", tokenizer=tokenizer,max_new_tokens=300)
print(generated_text)

generated_text = model.generate("Please tell me the truth", tokenizer=tokenizer,max_new_tokens=300)
print(generated_text)



